In [1]:
import numpy as np
import os
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import math
import random

MIN_LIST_LEN = 8
MAX_LIST_LEN = 8
MAX_STEPS = 210
SUCCESS_REWARD = 10.0
EPS_START = 0.5
EPS_END = 0.05
EPS_DECAY = 1000
GAMMA = 0.99
NUM_EPISODES = 100000
EPISODES_SAVE = 1000
OUTPUT_DIR = 'datasets/rl_sort_transformer_easy/list8_transformer3_128_gamma099_step210'

# Define the vocabulary
vocab = {
    'Comparison': 0,
    'Swap': 1,
    '0': 2,
    '1': 3,
    '2': 4,
    '3': 5,
    '4': 6,
    '5': 7,
    '6': 8,
    '7': 9,
    'less': 10,
    'equal': 11,
    'more': 12,
    'len1': 13,
    'len2': 14,
    'len3': 15,
    'len4': 16,
    'len5': 17,
    'len6': 18,
    'len7': 19,
    'len8': 20,
}
inv_vocab = {v: k for k, v in vocab.items()}

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

# Define the environment
class SortingEnv:
    def __init__(self):
        self.max_steps = MAX_STEPS

    def reset(self):
        self.length = random.randint(MIN_LIST_LEN, MAX_LIST_LEN)
        self.list = [random.randint(1, 100) for _ in range(self.length)]
        while self.list == sorted(self.list):
            self.list = [random.randint(1, 100) for _ in range(self.length)]
        self.indices = None
        self.current_step = 0
        self.done = False
        initial_token = 'len{}'.format(self.length)
        return vocab[initial_token], self.list.copy()
    
    def get_list(self):
        return self.list
    
    def get_list_len(self):
        return len(self.list)

    def step(self, action_tokens):
        action = action_tokens[0]
        reward = -0.01  # default penalty
        response_token = None

        if action == vocab['Comparison']:
            if len(action_tokens) != 3:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1 = action_tokens[1] - vocab['0']
            index2 = action_tokens[2] - vocab['0']
            if index1 >= self.length or index2 >= self.length or index1 < 0 or index2 < 0:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            self.indices = (index1, index2)
            if self.list[index1] < self.list[index2]:
                response_token = vocab['less']
                reward = -0.01
            elif self.list[index1] == self.list[index2]:
                response_token = vocab['equal']
                reward = -0.02
            else:
                response_token = vocab['more']
                reward = -0.01
        elif action == vocab['Swap']:
            if self.indices is None:
                reward = -1.0
                self.done = True
                return response_token, reward, self.done, self.list.copy()
            index1, index2 = self.indices
            prev_list = self.list.copy()
            self.list[index1], self.list[index2] = self.list[index2], self.list[index1]
            if self.list == sorted(self.list):
                reward = SUCCESS_REWARD
                self.done = True
            #elif prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]:
            #    reward = 0.1
            elif (index1 < index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]):
                reward = 0.1
            elif (index1 < index2 and prev_list[index1] < prev_list[index2] and self.list[index1] >= self.list[index2]) or \
                (index1 > index2 and prev_list[index1] > prev_list[index2] and self.list[index1] <= self.list[index2]):
                reward = -0.1
            else:
                reward = -0.01
            self.indices = None
        else:
            reward = -1.0
            self.done = True

        self.current_step += 1
        if self.current_step >= self.max_steps:
            self.done = True
        return response_token, reward, self.done, self.list.copy()


Using device: cuda


In [2]:
# Positional Encoding for Transformer
class PositionalEncoding(nn.Module):
    def __init__(self, d_model, max_len=256):
        super(PositionalEncoding, self).__init__()
        self.dropout = nn.Dropout(p=0.1)

        pe = torch.zeros(max_len, d_model)  # (max_len, d_model)
        position = torch.arange(0, max_len, dtype=torch.float).unsqueeze(1)  # (max_len, 1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() *
                             (-math.log(10000.0) / d_model))
        pe[:, 0::2] = torch.sin(position * div_term)  # Even indices
        pe[:, 1::2] = torch.cos(position * div_term)  # Odd indices
        pe = pe.unsqueeze(1)  # (max_len, 1, d_model)
        self.register_buffer('pe', pe)

    def forward(self, x):
        x = x + self.pe[:x.size(0)]
        return self.dropout(x)

# Transformer Model
class TransformerModel(nn.Module):
    def __init__(self, vocab_size, d_model=128, nhead=8, num_layers=3):
        super(TransformerModel, self).__init__()
        self.model_type = 'Transformer'
        self.d_model = d_model

        self.embedding = nn.Embedding(vocab_size, d_model)
        self.pos_encoder = PositionalEncoding(d_model)
        encoder_layers = nn.TransformerEncoderLayer(d_model, nhead)
        self.transformer_encoder = nn.TransformerEncoder(encoder_layers, num_layers)
        self.decoder = nn.Linear(d_model, vocab_size)

        self.init_weights()

    def init_weights(self):
        initrange = 0.1
        nn.init.uniform_(self.embedding.weight, -initrange, initrange)
        nn.init.zeros_(self.decoder.bias)
        nn.init.uniform_(self.decoder.weight, -initrange, initrange)

    def forward(self, src):
        src = self.embedding(src) * math.sqrt(self.d_model)
        src = self.pos_encoder(src)
        output = self.transformer_encoder(src)
        output = self.decoder(output)
        return output

def decode(input_tokens, inv_vocab):
    return ' '.join([inv_vocab[x] for x in input_tokens])


def save_checkpoint(model, optimizer, episode, folder, filename):
    """
    Save the model and optimizer state to the designated filepath.

    Args:
        model (nn.Module): The model to save.
        optimizer (torch.optim.Optimizer): The optimizer whose state to save.
        episode (int): The current episode number.
        filepath (str): The path where to save the checkpoint.
    """
    filepath = os.path.join(folder, filename)
    # Ensure the directory exists
    os.makedirs(os.path.dirname(filepath), exist_ok=True)
    # Save the checkpoint
    torch.save({
        'episode': episode,
        'model_state_dict': model.state_dict(),
        'optimizer_state_dict': optimizer.state_dict(),
    }, filepath)
    print(f"Checkpoint saved at episode {episode} to {filepath}")

def load_checkpoint(filepath, model, optimizer):
    """
    Load the model and optimizer state from the designated filepath.

    Args:
        filepath (str): The path from where to load the checkpoint.
        model (nn.Module): The model into which to load the state_dict.
        optimizer (torch.optim.Optimizer): The optimizer into which to load the state.

    Returns:
        int: The episode number to resume from.
    """
    if os.path.isfile(filepath):
        checkpoint = torch.load(filepath, map_location=device)
        model.load_state_dict(checkpoint['model_state_dict'])
        optimizer.load_state_dict(checkpoint['optimizer_state_dict'])
        episode = checkpoint['episode']
        print(f"Checkpoint loaded from {filepath}, resuming from episode {episode}")
        return episode
    else:
        print(f"No checkpoint found at {filepath}, starting from scratch.")
        return 0

In [ ]:
# Training Loop
def train(verbose=False):
    torch.autograd.set_detect_anomaly(True)  # Detect anomalies in autograd
    vocab_size = len(vocab)
    model = TransformerModel(vocab_size).to(device)
    optimizer = optim.Adam(model.parameters(), lr=1e-4)  # Reduced learning rate
    #load_checkpoint("datasets/rl_sort_transformer_easy/list6_transformer3_128_gamma098/ckpt_7000_0.9286.pth", model, optimizer)

    episode_cnt = 0
    total_reward = 0.0
    for episode in range(NUM_EPISODES):
        model.train()  # Set model to training mode
        env = SortingEnv()
        initial_token_id, current_list = env.reset()
        input_tokens = [initial_token_id]
        log_probs = []
        rewards = []
        state = 'expect_action'
        done = False

        while not done and len(input_tokens) < env.max_steps:
            if verbose:
                print(decode(input_tokens, inv_vocab))
                print(env.get_list())
            # Prepare input tensor
            input_seq = torch.tensor(input_tokens, dtype=torch.long, device=device).unsqueeze(1)  # (seq_len, batch_size)
            # Get model output
            output = model(input_seq)  # (seq_len, batch_size, vocab_size)
            # Get logits for the last token
            logits = output[-1, 0, :]  # (vocab_size)

            # Check for NaNs in logits
            if torch.isnan(logits).any():
                print(f"Episode {episode}, NaNs in logits before masking.")
                break

            # Get valid tokens based on state
            def get_valid_tokens(state):
                action_tokens = [vocab['Comparison'], vocab['Swap']]
                index_tokens = [vocab[str(i)] for i in range(env.length)]
                if state == 'expect_action':
                    return action_tokens
                elif state == 'expect_index1':
                    return index_tokens[:-1]
                elif state == 'expect_index2':
                    return [x for x in index_tokens if x > input_tokens[-1]]
                else:
                    # Handle unexpected states by defaulting to expect_action
                    return action_tokens

            valid_token_ids = get_valid_tokens(state)

            # Ensure valid_token_ids are within the vocab range
            if any(idx >= vocab_size or idx < 0 for idx in valid_token_ids):
                print(f"Episode {episode}, invalid indices in valid_token_ids: {valid_token_ids}")
                break

            # Mask invalid tokens
            mask_value = -1e9  # Use a large negative value instead of -inf
            mask = torch.full_like(logits, mask_value).to(device)
            mask[valid_token_ids] = 0
            masked_logits = logits + mask
            
            # Sample action. Have some chance to randomly pick a valid action.
            eps_threshold = EPS_END + (EPS_START - EPS_END) * np.exp(-1.0 * episode / EPS_DECAY)
            if random.random() < eps_threshold:
                masked_logits = masked_logits / 4

            # Check for NaNs in masked_logits
            if torch.isnan(masked_logits).any():
                print(f"Episode {episode}, NaNs in masked_logits after masking.")
                break

            # Compute probabilities
            probs = F.softmax(masked_logits, dim=0)

            # Check for NaNs in probs
            if torch.isnan(probs).any():
                print(f"Episode {episode}, NaNs in probs after softmax.")
                break

            try:
                m = torch.distributions.Categorical(probs)
                action_token = m.sample()
                log_prob = m.log_prob(action_token)
            except ValueError as e:
                print(f"Episode {episode}, error in sampling action: {e}")
                break

            log_probs.append(log_prob)
            input_tokens.append(action_token.item())

            action = action_token.item()
            reward = 0.0
            if state == 'expect_action':
                if action == vocab['Comparison']:
                    state = 'expect_index1'
                elif action == vocab['Swap']:
                    if env.indices is None:
                        # Can't perform Swap without a previous Comparison
                        reward = -1.0
                        rewards.append(reward)
                        done = True
                        continue
                    action_tokens = [vocab['Swap']]
                    response_token, reward, done, current_list = env.step(action_tokens)
                    if verbose:
                        print("Reward:", reward)
                    #rewards.append(reward)
                    state = 'expect_action'
                else:
                    # Invalid action, end the episode
                    reward = -1.0
                    #rewards.append(reward)
                    done = True
            elif state == 'expect_index1':
                index1_token = action_token
                state = 'expect_index2'
            elif state == 'expect_index2':
                index2_token = action_token
                action_tokens = [vocab['Comparison'], index1_token.item(), index2_token.item()]
                response_token, reward, done, current_list = env.step(action_tokens)
                if verbose:
                    print("Reward:", reward)
                #rewards.append(reward)
                if response_token is not None:
                    input_tokens.append(response_token)
                state = 'expect_action'
            else:
                # Invalid state, end the episode
                reward = -1.0
                #rewards.append(reward)
                done = True
            #
            rewards.append(reward)
        #
        # Save checkpoint
        if episode > 0 and episode % EPISODES_SAVE == 0:
            avg_reward = total_reward / (episode_cnt + 0.00001)
            episode_cnt = 0
            total_reward = 0.0
            save_checkpoint(model, optimizer, episode, OUTPUT_DIR, f"ckpt_{episode}_{avg_reward:.4f}.pth")
        #
        #print(decode(input_tokens, inv_vocab))
        #print(env.get_list())
        #print("rewards:", rewards)
        assert len(log_probs) == len(rewards), "log_probs and returns have different sizes!"
        
        if len(log_probs) == 0:
            continue  # Skip if no actions were taken

        # Compute returns
        returns = []
        R = 0
        gamma = GAMMA
        for r in rewards[::-1]:
            R = r + gamma * R
            returns.insert(0, R)
        returns = torch.tensor(returns)
        #print(returns)

        # Check for NaNs in returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in returns.")
            continue

#         if returns.std() != 0 and not torch.isnan(returns.std()):
#             returns = (returns - returns.mean()) / (returns.std() + 1e-9)
#         else:
#             pass #returns = returns - returns.mean()

        # Check for NaNs in standardized returns
        if torch.isnan(returns).any():
            print(f"Episode {episode}, NaNs in standardized returns.")
            continue

        # Compute loss
        loss = 0
        #print("log_probs:", log_probs)
        #print("returns:", returns)
        assert len(log_probs) == len(returns), "log_probs and returns have different sizes!"
        for log_prob, R in zip(log_probs, returns):
            loss -= log_prob * R

        # Check for NaNs in loss
        if torch.isnan(loss):
            print(f"Episode {episode}, NaN in loss.")
            continue

        optimizer.zero_grad()
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)  # Gradient clipping
        optimizer.step()

        episode_cnt += 1
        total_reward += sum(rewards)
        if episode % 1 == 0:
            print(f"Episode {episode}, loss: {loss.item():.4f}, total reward: {sum(rewards):.4f}")

if __name__ == "__main__":
    train(verbose=False)


/home/mcwave/.local/lib/python3.10/site-packages/torch/nn/modules/transformer.py:282: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(f"enable_nested_tensor is True, but self.use_nested_tensor is False because {why_not_sparsity_fast_path}")


Episode 0, loss: -23.7438, total reward: -0.9600
Episode 1, loss: -8.2705, total reward: -0.9200
Episode 2, loss: -49.2720, total reward: -0.8300
Episode 3, loss: -6.1235, total reward: -0.9200
Episode 4, loss: -25.5703, total reward: -1.0800
Episode 5, loss: -1.5408, total reward: -1.0000
Episode 6, loss: -12.4977, total reward: -0.9400
Episode 7, loss: -12.0577, total reward: -0.9300
Episode 8, loss: -25.3064, total reward: -1.2600
Episode 9, loss: -90.4774, total reward: -1.4700
Episode 10, loss: -0.9747, total reward: -1.0000
Episode 11, loss: -92.2992, total reward: -1.2600
Episode 12, loss: -1.2158, total reward: -1.0000
Episode 13, loss: -87.7318, total reward: -1.3500
Episode 14, loss: -85.9030, total reward: -1.3700
Episode 15, loss: -1.3806, total reward: -1.0000
Episode 16, loss: -40.0651, total reward: -0.8100
Episode 17, loss: -16.6363, total reward: -0.2100
Episode 18, loss: -36.2025, total reward: -0.6100
Episode 19, loss: -13.0180, total reward: -0.3100
Episode 20, loss

Episode 164, loss: -19.2539, total reward: -0.4000
Episode 165, loss: -19.6225, total reward: -0.4600
Episode 166, loss: -13.1475, total reward: -0.4000
Episode 167, loss: -6.5519, total reward: -0.2000
Episode 168, loss: -19.2145, total reward: -0.4200
Episode 169, loss: -61.9480, total reward: -1.4000
Episode 170, loss: -9.8491, total reward: -0.2100
Episode 171, loss: -2.7164, total reward: -1.0000
Episode 172, loss: -19.5869, total reward: -0.5200
Episode 173, loss: -16.3433, total reward: -0.3100
Episode 174, loss: -56.6991, total reward: -1.1700
Episode 175, loss: -11.8551, total reward: -0.3200
Episode 176, loss: -18.9831, total reward: -0.3000
Episode 177, loss: -30.8720, total reward: -0.6200
Episode 178, loss: -24.7453, total reward: -0.6200
Episode 179, loss: -17.1370, total reward: -0.4200
Episode 180, loss: -11.8952, total reward: -0.3200
Episode 181, loss: -17.2916, total reward: -0.3100
Episode 182, loss: -51.0275, total reward: -1.0100
Episode 183, loss: -29.2453, total

Episode 328, loss: 749.3007, total reward: 10.3300
Episode 329, loss: 7.5285, total reward: 0.2100
Episode 330, loss: 13.4468, total reward: 0.3100
Episode 331, loss: -66.4397, total reward: -0.4600
Episode 332, loss: -21.0749, total reward: -0.9700
Episode 333, loss: 9.1101, total reward: 0.4000
Episode 334, loss: 9.0945, total reward: 0.5900
Episode 335, loss: -33.0390, total reward: -0.8500
Episode 336, loss: 12.2236, total reward: 0.2000
Episode 337, loss: 709.4496, total reward: 10.4100
Episode 338, loss: -10.5209, total reward: -0.9300
Episode 339, loss: -5.1837, total reward: 0.2000
Episode 340, loss: 4.9529, total reward: 0.4000
Episode 341, loss: 3.8488, total reward: 0.3000
Episode 342, loss: 14.7049, total reward: 0.4200
Episode 343, loss: 15.5944, total reward: 0.6100
Episode 344, loss: -30.0694, total reward: -0.7300
Episode 345, loss: 25.1903, total reward: 0.4100
Episode 346, loss: 15.9257, total reward: 0.6100
Episode 347, loss: 1.4205, total reward: 0.4000
Episode 348,

Episode 494, loss: 576.0110, total reward: 10.3300
Episode 495, loss: 4.5627, total reward: 0.3000
Episode 496, loss: 3.2746, total reward: 0.1500
Episode 497, loss: -39.6313, total reward: -0.3800
Episode 498, loss: -5.3008, total reward: 0.5100
Episode 499, loss: -4.2608, total reward: -0.1000
Episode 500, loss: -21.9715, total reward: -0.6000
Episode 501, loss: 686.6964, total reward: 10.3000
Episode 502, loss: 678.8643, total reward: 11.0800
Episode 503, loss: -16.9917, total reward: -0.8500
Episode 504, loss: 23.0591, total reward: 1.0300
Episode 505, loss: -2.7500, total reward: 0.2000
Episode 506, loss: 2.3261, total reward: 0.2100
Episode 507, loss: 2.3443, total reward: 0.4100
Episode 508, loss: 2.6717, total reward: 0.2000
Episode 509, loss: 17.6787, total reward: 0.5000
Episode 510, loss: 3.4408, total reward: 0.4000
Episode 511, loss: -0.0421, total reward: 0.3000
Episode 512, loss: -11.3312, total reward: 0.0800
Episode 513, loss: 6.7249, total reward: 0.5000
Episode 514, 

...

Episode 9250, loss: 20.1411, total reward: 1.0200
Episode 9251, loss: 259.1786, total reward: 10.1800
Episode 9252, loss: 397.8271, total reward: 10.4100
Episode 9253, loss: 295.5074, total reward: 10.8900
Episode 9254, loss: 17.6052, total reward: 0.8100
Episode 9255, loss: 11.8931, total reward: 0.7100
Episode 9256, loss: 412.9704, total reward: 10.8500
Episode 9257, loss: 12.4560, total reward: 0.6100
Episode 9258, loss: 193.5136, total reward: 10.6000
Episode 9259, loss: 5.0137, total reward: 0.4000
Episode 9260, loss: 443.4702, total reward: 10.7100
Episode 9261, loss: 16.0245, total reward: 0.9100
Episode 9262, loss: 14.4974, total reward: 0.9100
Episode 9263, loss: 21.0261, total reward: 1.1200
Episode 9264, loss: 1.5900, total reward: 0.2000
Episode 9265, loss: 357.5265, total reward: 10.3000
Episode 9266, loss: 13.1261, total reward: 0.6900
Episode 9267, loss: 331.6938, total reward: 10.5400
Episode 9268, loss: 11.9518, total reward: 0.4000
Episode 9269, loss: 478.1418, total 

Episode 9413, loss: 377.8770, total reward: 10.4300
Episode 9414, loss: 12.0232, total reward: 0.6100
Episode 9415, loss: 19.0598, total reward: 0.9100
Episode 9416, loss: 397.4279, total reward: 11.1500
Episode 9417, loss: 17.3232, total reward: 0.6100
Episode 9418, loss: 364.9455, total reward: 10.1700
Episode 9419, loss: 339.3853, total reward: 10.7300
Episode 9420, loss: 27.7815, total reward: 1.2100
Episode 9421, loss: 20.3327, total reward: 1.0200
Episode 9422, loss: 329.9977, total reward: 10.6400
Episode 9423, loss: 15.7027, total reward: 1.2200
Episode 9424, loss: 22.6095, total reward: 0.8100
Episode 9425, loss: 20.6382, total reward: 1.2200
Episode 9426, loss: 493.1225, total reward: 11.1000
Episode 9427, loss: 417.7303, total reward: 11.0400
Episode 9428, loss: 11.0971, total reward: 0.8100
Episode 9429, loss: 5.5098, total reward: 0.1800
Episode 9430, loss: 13.9053, total reward: 0.7100
Episode 9431, loss: 397.3907, total reward: 11.0300
Episode 9432, loss: 405.8262, total

Episode 9575, loss: 315.6824, total reward: 10.4200
Episode 9576, loss: 16.3779, total reward: 0.9100
Episode 9577, loss: 12.0849, total reward: 0.8100
Episode 9578, loss: 295.2998, total reward: 10.4700
Episode 9579, loss: 360.8483, total reward: 10.2600
Episode 9580, loss: 382.2279, total reward: 10.4300
Episode 9581, loss: 8.0845, total reward: 0.8100
Episode 9582, loss: 18.5445, total reward: 0.9100
Episode 9583, loss: 10.7119, total reward: 1.0800
Episode 9584, loss: 20.0329, total reward: 1.0200
Episode 9585, loss: 23.8107, total reward: 1.1000
Episode 9586, loss: 352.5722, total reward: 10.9600
Episode 9587, loss: 500.5046, total reward: 10.8600
Episode 9588, loss: 302.1758, total reward: 10.6400
Episode 9589, loss: 27.6344, total reward: 1.3200
Episode 9590, loss: 28.7180, total reward: 1.2200
Episode 9591, loss: 434.5637, total reward: 11.0600
Episode 9592, loss: 397.9453, total reward: 11.0500
Episode 9593, loss: 13.4911, total reward: 0.8100
Episode 9594, loss: 2.8668, total

Episode 9737, loss: 7.5099, total reward: 0.5000
Episode 9738, loss: 391.6208, total reward: 10.7300
Episode 9739, loss: 419.0998, total reward: 10.9600
Episode 9740, loss: 297.7206, total reward: 10.6800
Episode 9741, loss: 21.4754, total reward: 1.0100
Episode 9742, loss: 288.3954, total reward: 10.5100
Episode 9743, loss: 2.9044, total reward: 0.4000
Episode 9744, loss: 28.8934, total reward: 1.2200
Episode 9745, loss: 8.4399, total reward: 0.8100
Episode 9746, loss: 0.8482, total reward: 0.0900
Episode 9747, loss: 12.3558, total reward: 0.7100
Episode 9748, loss: 14.6904, total reward: 0.8100
Episode 9749, loss: 280.3533, total reward: 10.4100
Episode 9750, loss: -5.9677, total reward: -0.0300
Episode 9751, loss: 3.3839, total reward: 0.3000
Episode 9752, loss: 22.5928, total reward: 1.0800
Episode 9753, loss: 378.2153, total reward: 10.4100
Episode 9754, loss: 352.2977, total reward: 10.7800
Episode 9755, loss: 7.1783, total reward: 0.5100
Episode 9756, loss: -2.5286, total reward

Episode 9899, loss: 2.8822, total reward: 0.4000
Episode 9900, loss: 6.7354, total reward: 0.4000
Episode 9901, loss: 14.0420, total reward: 1.1200
Episode 9902, loss: 16.5277, total reward: 0.9100
Episode 9903, loss: 5.2984, total reward: 0.5000
Episode 9904, loss: 22.1081, total reward: 1.1200
Episode 9905, loss: 9.8264, total reward: 0.6100
Episode 9906, loss: 432.0928, total reward: 11.1000
Episode 9907, loss: 1.8379, total reward: 0.3000
Episode 9908, loss: 267.5060, total reward: 10.2800
Episode 9909, loss: 5.0294, total reward: 0.4900
Episode 9910, loss: 12.8839, total reward: 0.7100
Episode 9911, loss: 13.0628, total reward: 0.8700
Episode 9912, loss: -1.5055, total reward: 0.4000
Episode 9913, loss: 344.6174, total reward: 10.3900
Episode 9914, loss: 400.6361, total reward: 10.3100
Episode 9915, loss: 11.0339, total reward: 0.7000
Episode 9916, loss: 308.0763, total reward: 10.5300
Episode 9917, loss: 12.8004, total reward: 1.0200
Episode 9918, loss: -4.1756, total reward: 0.0